# Data Analytics Capstone Project
### Beth Harvey
### October 22, 2023

# Introduction

As climate change, habitat loss, and disease continue to threaten wildlife throughout the world, detailed research is becoming more and more important. In 2023 alone, the U.S. Fish and Wildlife service took 21 species off of the Endangered Species Act list because they are now considered extinct [6]. Ten of those 21 species are birds. More research and resources are needed to develop effective conservation efforts. 

Some key components of bird conservation research is being able to accurately classify individual specimens, as well as being able to identify potential connections between different species. One of the most valuable resources for an effort like this is an extensive collection of detailed measurements for a wide variety of species.

There are several different families of birds that are difficult to differentiate. One of the more challenging pairs is old-world sparrows and new-world sparrows [2]. Both families are part of the order Passeriformes, but the old-world sparrows make up the Passeridae family, while the new-world sparrows compose the Passerellidae family [4] [3]. New-world sparrows are native to North America, while old-world sparrows are native to Europe, but are also widespread in North America as introduced species [2]. It is important to be able to distinguish if a specimen is an old-world or new-world sparrow before exploring conservation efforts, so that these efforts can focus on helping native species, not introduced ones.

## Project Goals

There are several different families of birds that are difficult to differentiate. One of the more challenging pairs is old-world sparrows and new-world sparrows. 
The end goal of this project is to build a model that can predict what taxonomic order a bird falls into based on several physical measurements. This could help researchers more accurately classify individuals when two or more species are possible. The model will be trained and tested using the AVONET dataset [8], a collective effort by researchers and volunteers to provide measurements of over 90,000 specimens of over 11,000 bird species. 

## Data

The data used for this project is from a large, collective effort known as the AVONET dataset. It is a collection of physical measurements, habitat information, and geographic information for over 90,000 specimens representing over 11,000 species of birds [3]. The majority of individuals measured were museum specimens, though many live birds were included as well. The raw data can be downloaded as Microsoft Excel files from https://figshare.com/s/b990722d72a26b5bfead [2]. The physical measurements are described in the Metadata sheet of Supplementary dataset 1 from the link above, and they are also shown in the figure below [3].

![Illustration of bird showing measurements used](measurements-fig.jpg)

The measurements are:

   1. Beak length from tip to base of skull
   2. Beak length from front edge of nostril to tip
   3. Beak width from front edges of nostrils
   4. Beak depth from front edges of nostrils
   5. Lower leg (tarsus) length
   6. Wing length from bend to end of longest feather 
   7. Wing length from bend to end of outer secondary (shorter) feather
   8. Length from tip of outer secondary (shorter) feather to tip of longest feather
   9. Tail length [3]


The remaining measurements are the mass of the bird and the "hand-wing index," which is calculated using measurements 6 and 8. Information about the bird's habitat, migration pattern(s), eating habits, conservation status, and geographic location are also included [2]. The physical measurements and age, sex, and country are all from the AVONET Raw Data sheet of Supplemental dataset 1, and the geographic and behavior information come from the AVONET3 BirdTree sheet of the same file [2]. The categorical information from the BirdTree sheet was matched to the appropriate raw data record to create the additional features. This sheet was chosen over the AVONET1 BirdLife and AVONET2 eBird sheets because it had a match for 90,303 of the 90,372 raw records, which was the highest of the three. The physical measurements shown in the figure above and the mass and hand-wing index are the key features used for this model.

In [1]:
# Import modules
import pandas as pd
from sklearn.impute import KNNImputer
import category_encoders as ce

In [2]:
# Import data
avonet_raw = pd.read_csv('avonet_raw.csv')
birdtree = pd.read_csv('avonet3_birdtree.csv')

# Data Inspection and Cleaning

### Merging Datasets

In [3]:
# Get desired additional features from BirdTree file
birdtree_features = birdtree[['Species3', 'Family3', 'Order3', 'Mass', 'Habitat', 'Habitat.Density', 'Migration', 
                              'Trophic.Level', 'Trophic.Niche','Primary.Lifestyle', 'Centroid.Latitude', 
                             'Centroid.Longitude']]

# Rename column to match avonet_raw column for merging
birdtree_features = birdtree_features.rename(columns = {'Species3': 'Species3_BirdTree', 'Family3': 'Family',
                                                       'Order3': 'Order'})

In [4]:
# Merge dataframes on Species3_BirdTree
avonet = pd.merge(avonet_raw, birdtree_features, how = 'inner', on = 'Species3_BirdTree')
avonet.head()

,Avibase.ID,Species1_BirdLife,Species2_eBird,eBird.species.group,Species3_BirdTree,Data.type,Source,Specimen.number,Sex,Age,...,Order,Mass,Habitat,Habitat.Density,Migration,Trophic.Level,Trophic.Niche,Primary.Lifestyle,Centroid.Latitude,Centroid.Longitude
0,AVIBASE-B3F5E5E2,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,1.0,NHMUK,11.4.1890,F,0.0,...,Apodiformes,2.7,Forest,2.0,1.0,Herbivore,Nectarivore,Aerial,15.29,-89.79
1,AVIBASE-B3F5E5E2,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,1.0,NHMUK,12.23.1890,F,0.0,...,Apodiformes,2.7,Forest,2.0,1.0,Herbivore,Nectarivore,Aerial,15.29,-89.79
2,AVIBASE-B3F5E5E2,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,1.0,NHMUK,1913.3.20.972,M,0.0,...,Apodiformes,2.7,Forest,2.0,1.0,Herbivore,Nectarivore,Aerial,15.29,-89.79
3,AVIBASE-B3F5E5E2,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,1.0,NHMUK,87.3.22.1574,M,0.0,...,Apodiformes,2.7,Forest,2.0,1.0,Herbivore,Nectarivore,Aerial,15.29,-89.79
4,AVIBASE-B3F5E5E2,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,Abeillia abeillei,1.0,NHMUK,NaN,M,0.0,...,Apodiformes,2.7,Forest,2.0,1.0,Herbivore,Nectarivore,Aerial,15.29,-89.79


In [5]:
# Get just the sparrow species (Passerellidae and Passeridae families)
sparrows_raw = avonet[avonet.Family.isin(['Passerellidae', 'Passeridae'])]
sparrows_raw.head()

,Avibase.ID,Species1_BirdLife,Species2_eBird,eBird.species.group,Species3_BirdTree,Data.type,Source,Specimen.number,Sex,Age,...,Order,Mass,Habitat,Habitat.Density,Migration,Trophic.Level,Trophic.Niche,Primary.Lifestyle,Centroid.Latitude,Centroid.Longitude
12842,AVIBASE-B1A5B22C,Carpospiza brachydactyla,Carpospiza brachydactyla,Carpospiza brachydactyla,Petronia brachydactyla,1.0,NHMUK,1934.1.1.8451,M,0.0,...,Passeriformes,22.3,Rock,3.0,3.0,Herbivore,Granivore,Terrestrial,32.97,52.87
12843,AVIBASE-B1A5B22C,Carpospiza brachydactyla,Carpospiza brachydactyla,Carpospiza brachydactyla,Petronia brachydactyla,1.0,NHMUK,1935.7.19.187,F,0.0,...,Passeriformes,22.3,Rock,3.0,3.0,Herbivore,Granivore,Terrestrial,32.97,52.87
12844,AVIBASE-B1A5B22C,Carpospiza brachydactyla,Carpospiza brachydactyla,Carpospiza brachydactyla,Petronia brachydactyla,1.0,NHMUK,1965.M.18659,F,0.0,...,Passeriformes,22.3,Rock,3.0,3.0,Herbivore,Granivore,Terrestrial,32.97,52.87
12845,AVIBASE-B1A5B22C,Carpospiza brachydactyla,Carpospiza brachydactyla,Carpospiza brachydactyla,Petronia brachydactyla,1.0,NHMUK,1975.10.188,M,0.0,...,Passeriformes,22.3,Rock,3.0,3.0,Herbivore,Granivore,Terrestrial,32.97,52.87
12846,AVIBASE-B1A5B22C,Carpospiza brachydactyla,Carpospiza brachydactyla,Carpospiza brachydactyla,Petronia brachydactyla,1.0,NHMUK,1934.9.20.459,M,0.0,...,Passeriformes,22.3,Rock,3.0,3.0,Herbivore,Granivore,Terrestrial,32.97,52.87


In [6]:
# Drop irrelevant columns
sparrows = sparrows_raw.drop(columns = ['Avibase.ID', 'Species1_BirdLife', 'Species2_eBird', 'eBird.species.group', 
                                    'Species3_BirdTree', 'Data.type', 'Source', 'Specimen.number', 'Locality', 
                                    'Country_WRI','Country', 'Measurer', 'Protocol', 'Publication', 'Order'], axis = 1)

In [7]:
sparrows.head()

,Sex,Age,Beak.Length_Culmen,Beak.Length_Nares,Beak.Width,Beak.Depth,Tarsus.Length,Wing.Length,Kipps.Distance,Secondary1,...,Family,Mass,Habitat,Habitat.Density,Migration,Trophic.Level,Trophic.Niche,Primary.Lifestyle,Centroid.Latitude,Centroid.Longitude
12842,M,0.0,13.8,8.8,5.2,6.8,18.3,93.0,38.0,55.0,...,Passeridae,22.3,Rock,3.0,3.0,Herbivore,Granivore,Terrestrial,32.97,52.87
12843,F,0.0,13.3,8.7,5.0,6.6,16.8,94.0,36.9,57.1,...,Passeridae,22.3,Rock,3.0,3.0,Herbivore,Granivore,Terrestrial,32.97,52.87
12844,F,0.0,13.9,8.1,5.8,7.5,18.3,91.0,37.4,53.6,...,Passeridae,22.3,Rock,3.0,3.0,Herbivore,Granivore,Terrestrial,32.97,52.87
12845,M,0.0,14.2,8.9,5.4,6.8,18.8,92.0,37.9,54.1,...,Passeridae,22.3,Rock,3.0,3.0,Herbivore,Granivore,Terrestrial,32.97,52.87
12846,M,0.0,15.2,NaN,6.1,6.8,18.5,98.0,NaN,NaN,...,Passeridae,22.3,Rock,3.0,3.0,Herbivore,Granivore,Terrestrial,32.97,52.87


### Inspect Dataset

In [8]:
# View shape and size of dataset
sparrows.shape

(380, 22)

In [9]:
sparrows.describe(include='all')

,Sex,Age,Beak.Length_Culmen,Beak.Length_Nares,Beak.Width,Beak.Depth,Tarsus.Length,Wing.Length,Kipps.Distance,Secondary1,...,Family,Mass,Habitat,Habitat.Density,Migration,Trophic.Level,Trophic.Niche,Primary.Lifestyle,Centroid.Latitude,Centroid.Longitude
count,380,380.000000,317.000000,233.000000,273.000000,286.000000,315.000000,380.000000,307.000000,307.000000,...,380,380.000000,369,380.000000,380.000000,380,380,380,380.000000,380.000000
unique,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,7,NaN,NaN,2,2,2,NaN,NaN
top,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Passeridae,NaN,Woodland,NaN,NaN,Herbivore,Granivore,Terrestrial,NaN,NaN
freq,210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,380,NaN,102,NaN,NaN,280,334,361,NaN,NaN
mean,NaN,0.034211,14.109148,9.601717,6.200000,7.483566,18.663810,79.606316,20.294788,60.489251,...,NaN,25.903947,NaN,2.563158,1.147368,NaN,NaN,NaN,22.393474,52.183868
std,NaN,0.182009,2.329111,1.560724,0.924622,1.003665,2.396792,14.779939,8.849817,9.755762,...,NaN,8.143272,NaN,0.496649,0.390343,NaN,NaN,NaN,22.518687,27.250055
min,NaN,0.000000,10.000000,6.400000,4.400000,5.000000,12.700000,54.000000,2.900000,44.000000,...,NaN,11.200000,NaN,2.000000,1.000000,NaN,NaN,NaN,-27.280000,-23.930000
25%,NaN,0.000000,12.700000,8.600000,5.500000,6.700000,16.900000,69.000000,15.050000,54.450000,...,NaN,21.400000,NaN,2.000000,1.000000,NaN,NaN,NaN,6.340000,34.840000
50%,NaN,0.000000,13.900000,9.400000,6.000000,7.400000,18.600000,76.000000,18.500000,58.000000,...,NaN,24.200000,NaN,3.000000,1.000000,NaN,NaN,NaN,29.070000,47.540000
75%,NaN,0.000000,15.000000,10.300000,6.900000,8.175000,20.000000,88.000000,22.500000,65.250000,...,NaN,29.050000,NaN,3.000000,1.000000,NaN,NaN,NaN,45.730000,76.630000


In [10]:
sparrows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 380 entries, 12842 to 77214
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Sex                 380 non-null    object 
 1   Age                 380 non-null    float64
 2   Beak.Length_Culmen  317 non-null    float64
 3   Beak.Length_Nares   233 non-null    float64
 4   Beak.Width          273 non-null    float64
 5   Beak.Depth          286 non-null    float64
 6   Tarsus.Length       315 non-null    float64
 7   Wing.Length         380 non-null    float64
 8   Kipps.Distance      307 non-null    float64
 9   Secondary1          307 non-null    float64
 10  Hand-wing.Index     307 non-null    float64
 11  Tail.Length         321 non-null    float64
 12  Family              380 non-null    object 
 13  Mass                380 non-null    float64
 14  Habitat             369 non-null    object 
 15  Habitat.Density     380 non-null    float64
 16  Migrati

### Cleaning Data

In [11]:
columns = sparrows.columns

# Determine percent of missing data in each column
# Drop columns with 30% or more missing data
for i in columns:
    missing = sparrows[i].isnull().sum()/len(sparrows)
    if missing >= 0.3:
        sparrows.drop(i, axis = 1, inplace = True)

In [12]:
sparrows.columns

Index(['Sex', 'Age', 'Beak.Length_Culmen', 'Beak.Width', 'Beak.Depth',
       'Tarsus.Length', 'Wing.Length', 'Kipps.Distance', 'Secondary1',
       'Hand-wing.Index', 'Tail.Length', 'Family', 'Mass', 'Habitat',
       'Habitat.Density', 'Migration', 'Trophic.Level', 'Trophic.Niche',
       'Primary.Lifestyle', 'Centroid.Latitude', 'Centroid.Longitude'],
      dtype='object')

In [13]:
# Drop rows with missing Habitat data
# There are very few missing values for this feature, so it won't decrease the dataset much
sparrows.dropna(subset=['Habitat'], inplace = True, 
              ignore_index = True)

In [14]:
# Impute missing numeric values using KNN imputation
# Use only numeric features with missing values
sparrows_missing = sparrows[['Beak.Length_Culmen', 'Beak.Width', 'Beak.Depth', 'Tarsus.Length', 'Wing.Length',
                         'Kipps.Distance', 'Secondary1', 'Hand-wing.Index', 'Tail.Length']]

# Create imputer object
knn_imputer = KNNImputer(n_neighbors = 5)

# Fit and transform numeric data with imputer
sparrows_num_trans = knn_imputer.fit_transform(sparrows_missing)

In [15]:
# Convert transformed numeric data to DataFrame
sparrows_num_df = pd.DataFrame(sparrows_num_trans, columns = ['Beak.Length_Culmen', 'Beak.Width', 'Beak.Depth', 
                                                          'Tarsus.Length', 'Wing.Length', 'Kipps.Distance', 
                                                              'Secondary1', 'Hand-wing.Index', 'Tail.Length'])
sparrows_num_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Beak.Length_Culmen  369 non-null    float64
 1   Beak.Width          369 non-null    float64
 2   Beak.Depth          369 non-null    float64
 3   Tarsus.Length       369 non-null    float64
 4   Wing.Length         369 non-null    float64
 5   Kipps.Distance      369 non-null    float64
 6   Secondary1          369 non-null    float64
 7   Hand-wing.Index     369 non-null    float64
 8   Tail.Length         369 non-null    float64
dtypes: float64(9)
memory usage: 26.1 KB


In [16]:
# Replace original columns with imputed columns
col_names = ['Beak.Length_Culmen', 'Beak.Width', 'Beak.Depth', 'Tarsus.Length', 'Wing.Length', 'Kipps.Distance', 
             'Secondary1', 'Hand-wing.Index', 'Tail.Length']

for col in col_names:
    sparrows[col] = sparrows_num_df[col]

In [17]:
# Change data type of Age, Habitat.Density and Migration to int
feature_list = ['Age', 'Habitat.Density', 'Migration']

for feature in feature_list:
    sparrows[feature] = sparrows[feature].astype(int)

In [18]:
sparrows.columns

Index(['Sex', 'Age', 'Beak.Length_Culmen', 'Beak.Width', 'Beak.Depth',
       'Tarsus.Length', 'Wing.Length', 'Kipps.Distance', 'Secondary1',
       'Hand-wing.Index', 'Tail.Length', 'Family', 'Mass', 'Habitat',
       'Habitat.Density', 'Migration', 'Trophic.Level', 'Trophic.Niche',
       'Primary.Lifestyle', 'Centroid.Latitude', 'Centroid.Longitude'],
      dtype='object')

In [19]:
# Rename columns to all lowercase, no periods
sparrows.columns = ['sex', 'age', 'beak_length_culmens', 'beak_width', 'beak_depth', 'tarsus_length', 'wing_length',
                  'kipps_dist', 'secondary1', 'hand_wing', 'tail_length', 'family', 'mass', 'habitat', 
                    'habitat_density', 'migration', 'trophic_level', 'trophic_niche', 'primary_lifestyle', 
                    'centroid_lat', 'centroid_long']

In [20]:
# View and inspect final dataset
sparrows.head()

,sex,age,beak_length_culmens,beak_width,beak_depth,tarsus_length,wing_length,kipps_dist,secondary1,hand_wing,...,family,mass,habitat,habitat_density,migration,trophic_level,trophic_niche,primary_lifestyle,centroid_lat,centroid_long
0,M,0,13.8,5.2,6.8,18.3,93.0,38.0,55.0,40.90,...,Passeridae,22.3,Rock,3,3,Herbivore,Granivore,Terrestrial,32.97,52.87
1,F,0,13.3,5.0,6.6,16.8,94.0,36.9,57.1,39.30,...,Passeridae,22.3,Rock,3,3,Herbivore,Granivore,Terrestrial,32.97,52.87
2,F,0,13.9,5.8,7.5,18.3,91.0,37.4,53.6,41.10,...,Passeridae,22.3,Rock,3,3,Herbivore,Granivore,Terrestrial,32.97,52.87
3,M,0,14.2,5.4,6.8,18.8,92.0,37.9,54.1,41.20,...,Passeridae,22.3,Rock,3,3,Herbivore,Granivore,Terrestrial,32.97,52.87
4,M,0,15.2,6.1,6.8,18.5,98.0,23.9,72.3,24.88,...,Passeridae,22.3,Rock,3,3,Herbivore,Granivore,Terrestrial,32.97,52.87


In [24]:
sparrows.tail_length

0      56.0
1      51.0
2      53.0
3      52.0
4      69.0
       ... 
364    41.0
365    39.0
366    50.0
367    40.0
368    45.0
Name: tail_length, Length: 369, dtype: float64

In [21]:
sparrows.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   sex                  369 non-null    object 
 1   age                  369 non-null    int64  
 2   beak_length_culmens  369 non-null    float64
 3   beak_width           369 non-null    float64
 4   beak_depth           369 non-null    float64
 5   tarsus_length        369 non-null    float64
 6   wing_length          369 non-null    float64
 7   kipps_dist           369 non-null    float64
 8   secondary1           369 non-null    float64
 9   hand_wing            369 non-null    float64
 10  tail_length          369 non-null    float64
 11  family               369 non-null    object 
 12  mass                 369 non-null    float64
 13  habitat              369 non-null    object 
 14  habitat_density      369 non-null    int64  
 15  migration            369 non-null    int

In [22]:
sparrows.describe(include = 'all')

,sex,age,beak_length_culmens,beak_width,beak_depth,tarsus_length,wing_length,kipps_dist,secondary1,hand_wing,...,family,mass,habitat,habitat_density,migration,trophic_level,trophic_niche,primary_lifestyle,centroid_lat,centroid_long
count,369,369.000000,369.000000,369.000000,369.000000,369.00000,369.000000,369.000000,369.000000,369.00000,...,369,369.000000,369,369.000000,369.000000,369,369,369,369.000000,369.000000
unique,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,7,NaN,NaN,2,2,2,NaN,NaN
top,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Passeridae,NaN,Woodland,NaN,NaN,Herbivore,Granivore,Terrestrial,NaN,NaN
freq,204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,369,NaN,102,NaN,NaN,269,323,350,NaN,NaN
mean,NaN,0.035230,14.109864,6.240271,7.541192,18.67252,79.858808,20.158537,59.670732,24.61290,...,NaN,25.975881,NaN,2.555556,1.151762,NaN,NaN,NaN,22.787696,52.709431
std,NaN,0.184612,2.211310,0.852006,0.953012,2.26727,14.920992,8.490963,9.482009,6.63041,...,NaN,8.235098,NaN,0.497579,0.395289,NaN,NaN,NaN,22.724446,27.425750
min,NaN,0.000000,10.000000,4.400000,5.000000,12.70000,54.000000,2.900000,44.000000,5.30000,...,NaN,11.200000,NaN,2.000000,1.000000,NaN,NaN,NaN,-27.280000,-23.930000
25%,NaN,0.000000,12.800000,5.680000,6.800000,16.96000,69.000000,15.000000,53.000000,20.48000,...,NaN,21.400000,NaN,2.000000,1.000000,NaN,NaN,NaN,6.340000,34.840000
50%,NaN,0.000000,13.900000,6.100000,7.500000,18.64000,76.500000,18.600000,57.500000,24.50000,...,NaN,24.200000,NaN,3.000000,1.000000,NaN,NaN,NaN,30.870000,52.870000
75%,NaN,0.000000,15.000000,6.800000,8.200000,20.00000,88.000000,22.000000,64.000000,27.60000,...,NaN,29.500000,NaN,3.000000,1.000000,NaN,NaN,NaN,45.730000,76.630000


### References

1. Bhattarai, A., T., F.: Knnimputer. https://github.com/scikit-learn/
scikit-learn/blob/093e0cf14/sklearn/impute/_knn.py#L20 (2014), accessed
on October 31, 2023
2. Camfield, A.: Emberizidae buntings, american sparrows, and relatives. https://
animaldiversity.org/accounts/Emberizidae/ (2004), accessed on November 6,
2023
3. Cornell: New world sparrows—passerellidae. https://www.allaboutbirds.org/
guide/browse/taxonomy/Passerellidae (2023), accessed on November 6, 2023
4. Cornell: Old world sparrows—passeridae. https://www.allaboutbirds.org/
guide/browse/taxonomy/Passeridae (2023), accessed on November 6, 2023
5. Pedregosa, F., Varoquaux, G., Gramfort, A., Michel, V., Thirion, B., Grisel, O.,
Blondel, M., Prettenhofer, P., Weiss, R., Dubourg, V., Vanderplas, J., Passos, A.,
Cournapeau, D., Brucher, M., Perrot, M., Duchesnay, E.: Scikit-learn: Machine
learning in Python. Journal of Machine Learning Research 12, 2825–2830 (2011)
6. Schuldheisz, C.: Fish and wildlife service delists 21 species from the endangered
species act due to extinction. https://www.fws.gov/press-release/2023-10/
21-species-delisted-endangered-species-act-due-extinction (2023), ac-
cessed on October 18, 2023
7. Tobias: Avonet: morphological, ecological and geographical data for all birds. https:
//figshare.com/s/b990722d72a26b5bfead (2022), accessed on October 18, 2023
8. Tobias, J.A., Sheard, C., Pigot, A.L., Devenish, A.J.M., Yang, J., Sayol, F., Neate-
Clegg, M.H.C., Alioravainen, N., Weeks, T.L., Barber, R.A., Walkden, P.A., Mac-
Gregor, H.E.A., Jones, S.E.I., Vincent, C., Phillips, A.G., Marples, N.M., Monta ̃no-
Centellas, F.A., Leandro-Silva, V., Claramunt, S., Darski, B., Freeman, B.G., Breg-
man, T.P., Cooney, C.R., Hughes, E.C., Capp, E.J.R., Varley, Z.K., Friedman,
N.R., Korntheuer, H., Corrales-Vargas, A., Trisos, C.H., Weeks, B.C., Hanz, D.M.,
T ̈opfer, T., Bravo, G.A., Remeˇs, V., Nowak, L., Carneiro, L.S., Moncada R., A.J.,
Matysiokov ́a, B., Baldassarre, D.T., Mart ́ınez-Salinas, A., Wolfe, J.D., Chapman,
P.M., Daly, B.G., Sorensen, M.C., Neu, A., Ford, M.A., Mayhew, R.J., Fabio Sil-
veira, L., Kelly, D.J., Annorbah, N.N.D., Pollock, H.S., Grabowska-Zhang, A.M.,
McEntee, J.P., Carlos T. Gonzalez, J., Meneses, C.G., Mu ̃noz, M.C., Powell, L.L.,
Jamie, G.A., Matthews, T.J., Johnson, O., Brito, G.R.R., Zyskowski, K., Crates,
R., Harvey, M.G., Jurado Zevallos, M., Hosner, P.A., Bradfer-Lawrence, T., Maley,
J.M., Stiles, F.G., Lima, H.S., Provost, K.L., Chibesa, M., Mashao, M., Howard,
J.T., Mlamba, E., Chua, M.A.H., Li, B., G ́omez, M.I., Garc ́ıa, N.C., P ̈ackert,
M., Fuchs, J., Ali, J.R., Derryberry, E.P., Carlson, M.L., Urriza, R.C., Brzeski,
K.E., Prawiradilaga, D.M., Rayner, M.J., Miller, E.T., Bowie, R.C.K., Lafontaine,
R.M., Scofield, R.P., Lou, Y., Somarathna, L., Lepage, D., Illif, M., Neuschulz,
E.L., Templin, M., Dehling, D.M., Cooper, J.C., Pauwels, O.S.G., Analuddin, K.,
Fjelds ̊a, J., Seddon, N., Sweet, P.R., DeClerck, F.A.J., Naka, L.N., Brawn, J.D.,
Aleixo, A., B ̈ohning-Gaese, K., Rahbek, C., Fritz, S.A., Thomas, G.H., Schleuning,
M.: Avonet: morphological, ecological and geographical data for all birds. Ecology
Bird Identification Model 7
Letters 25(3), 581–597 (2022). https://doi.org/https://doi.org/10.1111/ele.13898,
https://onlinelibrary.wiley.com/doi/abs/10.1111/ele.13898


In [23]:
# Display system data
from watermark import watermark
print(watermark())

Last updated: 2023-11-05T12:43:46.361708-06:00

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.2.0

Compiler    : Clang 12.0.0 
OS          : Darwin
Release     : 23.0.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

